In [77]:
%reset -f
import altair as alt
import pandas as pd

# Altair testing
stickers_df = pd.read_csv('stickers_df.csv')
stickers_df = stickers_df.merge(stickers_df[['sticker_url']].drop_duplicates().reset_index(), on='sticker_url')
stickers_df.rename(columns={'from':'sender','from_id':'total_count'}, inplace=True)
stickers_df.sort_values('total_count',ascending=False,inplace=True)

grouped_sticker_df = stickers_df.pivot(index='sticker_url',columns='sender',values='total_count').reset_index()
grouped_sticker_df['total'] = grouped_sticker_df['🐑💛🌙'] + grouped_sticker_df['🐥💛🌻']
grouped_sticker_df = grouped_sticker_df.sort_values(['total'], ascending=False).head(10)
grouped_sticker_df['🐑💛🌙'] = grouped_sticker_df['🐑💛🌙'].astype(int)
grouped_sticker_df['🐥💛🌻'] = grouped_sticker_df['🐥💛🌻'].astype(int)
grouped_sticker_df['total'] = grouped_sticker_df['total'].astype(int)

In [76]:
input_dropdown = alt.binding_select(options=['🐑💛🌙','🐥💛🌻'])
selection = alt.selection_single(fields=['sender'], bind=input_dropdown, name='From')
color = alt.condition(selection,
                    alt.Color('sender:N', legend=None),
                    alt.value('lightgray'))

img = alt.Chart(stickers_df).transform_filter(
    selection
).mark_image().encode(
    y=alt.Y('index:N', axis=alt.Axis(domainOpacity=0,ticks=False)),
    url="sticker_url:N"
).transform_window(
    rank='rank(total_count)',
    sort=[alt.SortField('total_count', order='descending')]
).transform_filter(
    (alt.datum.rank < 10)
).properties(height=300)

chart = alt.Chart(stickers_df).add_selection(
    selection
).transform_filter(
    selection
).mark_bar().encode(
    y=alt.Y('index:N', axis=None),
    x=alt.X('total_count:Q'),
    text='total_count:Q'
).transform_window(
    rank='rank(total_count)',
    sort=[alt.SortField('total_count', order='descending')]
).transform_filter(
    (alt.datum.rank < 10)
).properties(height=300)


alt.concat(
    img, chart
).configure_concat(
    spacing=0
)

alt.ConcatChart(...)